# 0. Prepare

In [1]:
import sys
sys.path.append('../../dataset/')
sys.path.append('../../network/')
sys.path.append('../../model/')

import os
import glob
import time
import torch
import joblib
import logging
import argparse
import numpy as np
import pandas as pd
import torch.optim as optim
import torch.nn as nn
import torch
import seaborn as sns

from pathlib import Path
from main_loading import *
from main_network import *
from main_model_rec import *
from main_model_one_class import *
from scipy.spatial import KDTree
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import NearestNeighbors

import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

In [3]:
device = 'cuda:1'
root = '/net/leksai/data/FashionMNIST'
rec_model_path = '/net/leksai/nips/model/rec/fmnist/rec_unsupervised_[0]_[]_[0.0]/net_fmnist_LeNet_rec_eta_100_epochs_150_batch_128/model.tar'
oc_model_path = '/net/leksai/nips/model/one_class/fmnist/one_class_unsupervised_[0]_[]_[1]_[0.0]/net_fmnist_LeNet_one_class_eta_100_epochs_150_batch_128/model.tar'

# 1. Load Only the Encoder Part

> ## For One-Class Model

In [4]:
class OneClassEncoder:
    def __init__(self):
        self.net = None
        self.net_name = None

    def set_network(self, net_name):
        self.net_name = net_name
        self.net = build_network(net_name)

    def load_model(self, model_path, map_location):
        model_dict = torch.load(model_path, map_location=map_location)
        self.c = model_dict['c']
        self.net.load_state_dict(model_dict['net_dict'])

    def test(self, train, dataset, device, batch_size, n_jobs_dataloader):
        if train:
            all_loader, _ = dataset.loaders(batch_size=batch_size,
                                            num_workers=n_jobs_dataloader)
        else:
            all_loader = dataset.loaders(batch_size=batch_size,
                                         num_workers=n_jobs_dataloader)
        net = self.net.to(device)
        criterion = nn.MSELoss(reduction='none')
        
        n_batches = 0
        X_pred_list = []
        net.eval()
        
        with torch.no_grad():
            for data in all_loader:
                X, y, idx = data
                X, y, idx = X.to(device), y.to(device), idx.to(device)

                X_pred = net(X)
                X_pred_list += X_pred
        
        return np.array(X_pred_list)

In [5]:
oc_encoder = OneClassEncoder()
oc_encoder.set_network('fmnist_LeNet_one_class')
oc_encoder.load_model(oc_model_path, device)

> ## For Reconstruction Model

In [6]:
class RecEncoder:
    def __init__(self):

        self.net_name = None
        self.net = None
        self.ae_net = None


    def set_network(self, net_name: str='fmnist_LeNet_one_class'):
        """
        Set the network structure for the model.
        The key here is to initialize <self.net>.
        """
        self.net_name = net_name
        self.net = build_network(net_name)
        self.ae_net = build_network('fmnist_LeNet_rec')

    def load_model(self,
                   model_path,
                   map_location='cuda:1'):
        """
        Load the trained model for the model.
        The key here is to initialize <self.c>.
        """
        # Load the general model
        model_dict = torch.load(model_path, map_location=map_location)
        self.ae_net.load_state_dict(model_dict['net_dict'])
        
        # Obtain the net dictionary
        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict()
        
        # Filter out decoder network keys
        ae_net_dict = {k: v for k, v in ae_net_dict.items() if k in net_dict}
        
        # Overwrite values in the existing state_dict
        net_dict.update(ae_net_dict)

        # Load the new state_dict
        self.net.load_state_dict(net_dict)
        

    def save_model(self, export_model, save_ae=True):
        net_dict = self.net.state_dict()
        torch.save({'net_dict': net_dict}, export_model)
    
    def test(self, train, dataset, device, batch_size, n_jobs_dataloader):
        if train:
            all_loader, _ = dataset.loaders(batch_size=batch_size,
                                            num_workers=n_jobs_dataloader)
        else:
            all_loader = dataset.loaders(batch_size=batch_size,
                                         num_workers=n_jobs_dataloader)
        net = self.net.to(device)
        criterion = nn.MSELoss(reduction='none')
        
        n_batches = 0
        X_pred_list = []
        net.eval()
        
        with torch.no_grad():
            for data in all_loader:
                X, y, idx = data
                X, y, idx = X.to(device), y.to(device), idx.to(device)

                X_pred = net(X)
                X_pred_list += X_pred
        
        return np.array(X_pred_list)

In [7]:
rec_encoder = RecEncoder()
rec_encoder.set_network()
rec_encoder.load_model(rec_model_path, device)

# 2. Dataset Loading

In [21]:
dataset_dict_train = {}
dataset_dict_test = {}
name_list = ['tshirt', 'trouser', 'pullover', 'dress', 'coat',
             'sandal', 'shirt', 'sneaker', 'bag', 'boot']

In [22]:
for i, name in enumerate(name_list):
    dataset_dict_train[name] = load_dataset(loader_name='fmnist',
                                            root=root,
                                            label_normal=(i,),
                                            ratio_abnormal=0)

Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!
Loading dataset for you!
Almost loaded!


In [23]:
for i, name in enumerate(name_list):
    dataset_dict_test[name] = load_dataset(loader_name='fmnist_eval',
                                            root=root,
                                            label_eval=(i,),
                                            test_eval=True)

# 3. Latent Vector Obtaining

> Let's use the unsupervised model of `pullover` as feature extractor, meaning that the latent space are defined from the neural weights of the unsupervised model of `pullover`.

In [24]:
latent_dict_train = {'oc':{}, 'rec':{}}
latent_dict_test = {'oc':{}, 'rec':{}}

> ## For One Class Model

In [70]:
for name in name_list:
    dataset_train = dataset_dict_train[name]
    data_train = oc_encoder.test(True, dataset_train, device, 6000, 0)
    data_train = np.array([x.cpu().numpy() for x in data_train])
    latent_dict_train['oc'][name] = data_train

In [71]:
for name in name_list:
    dataset_test = dataset_dict_test[name]
    data_test = oc_encoder.test(False, dataset_test, device, 1000, 0)
    data_test = np.array([x.cpu().numpy() for x in data_test])
    latent_dict_test['oc'][name] = data_test

> ## For Rec Model

In [28]:
for name in name_list:
    dataset_train = dataset_dict_train[name]
    data_train = rec_encoder.test(True, dataset_train, device, 6000, 0)
    data_train = np.array([x.cpu().numpy() for x in data_train])
    latent_dict_train['rec'][name] = data_train

In [27]:
for name in name_list:
    dataset_test = dataset_dict_test[name]
    data_test = rec_encoder.test(False, dataset_test, device, 1000, 0)
    data_test = np.array([x.cpu().numpy() for x in data_test])
    latent_dict_test['rec'][name] = data_test

# Function for KL Divergence by KNN

> Cited from https://github.com/nhartland/KL-divergence-estimators

In [29]:
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import KDTree

def knn_distance(point, sample, k):
    """ 
    Euclidean distance from `point` to it's `k`-Nearest
    Neighbour in `sample` 
    """
    norms = np.linalg.norm(sample-point, axis=1)
    return np.sort(norms)[k]


def verify_sample_shapes(s1, s2, k):
    # Expects [N, D]
    assert(len(s1.shape) == len(s2.shape) == 2)
    # Check dimensionality of sample is identical
    assert(s1.shape[1] == s2.shape[1])
    
    
def skl_estimator(s1, s2, k=3):
    """ 
    KL-Divergence estimator using scikit-learn's NearestNeighbours.
    Inputs:
        s1: (N_1,D) Sample drawn from distribution P
        s2: (N_2,D) Sample drawn from distribution Q
        k: Number of neighbours considered (default 1)
    return: 
        estimated D(P|Q)
    """
    verify_sample_shapes(s1, s2, k)

    n, m = len(s1), len(s2)
    d = float(s1.shape[1])
    D = np.log(m / (n - 1))

    s1_neighbourhood = NearestNeighbors(k + 1, 10).fit(s1)
    s2_neighbourhood = NearestNeighbors(k, 10).fit(s2)

    for p1 in s1:
        s1_distances, indices = s1_neighbourhood.kneighbors([p1], k + 1)
        s2_distances, indices = s2_neighbourhood.kneighbors([p1], k)
        rho = s1_distances[0][- 1]
        nu = s2_distances[0][- 1]
        D += (d / n) * np.log(nu / rho)
        D += 0
    return D

# Calculating Joint KL Divergence for Reconstruction Model

In [62]:
div_joint = {k: {} for k in name_list[1:]}
div_margin = {k: {} for k in name_list[1:]}

In [90]:
div_margin_train = {k: {} for k in name_list[1:]}

In [63]:
# Fix normal train, normal test.
normal_train = latent_dict_train['rec']['tshirt']
normal_test = latent_dict_test['rec']['tshirt']

In [64]:
# Fix abnormal test
# abnormal_test = latent_dict_test['rec']['shirt']
# div_joint_test = div_joint['shirt']
# div_margin_test = div_margin['shirt']

## Calculate for $KL(P(normal train) & P(abnormal train i) || P(normal test) & P(abnormal test))$

In [91]:
name_list

['tshirt',
 'trouser',
 'pullover',
 'dress',
 'coat',
 'sandal',
 'shirt',
 'sneaker',
 'bag',
 'boot']

In [66]:
for test_j in name_list[1:]:
    print('Outer loop', test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['rec'][test_j]
    div_joint_test = div_joint[test_j]
    div_margin_test = div_margin[test_j]
    
    for i in name_list[1:]:
        print(i)
        abnormal_train_i = latent_dict_train['rec'][i]

        # Calculating for joint divergence
        left_joint = np.r_[normal_train, abnormal_train_i]
        right_joint = np.r_[normal_test, abnormal_test]
        div_joint_test[i] = .5 * (skl_estimator(left_joint, right_joint) + 
                                  skl_estimator(right_joint, left_joint))
        print('KL joint:', div_joint_test[i])

        # Calculating for marginal divergence
        left_margin = normal_train
        right_margin = abnormal_train_i
        div_margin_test[i] = .5 * (skl_estimator(left_margin, right_margin) + 
                                   skl_estimator(right_margin, left_margin))
        print('KL marginal:', div_margin_test[i]) 

Outer loop trouser
trouser
KL joint: -0.01998289193292102
KL marginal: 41.86391770552747
pullover
KL joint: 16.719424922947773
KL marginal: 26.21745876800793
dress
KL joint: 11.818667496670319
KL marginal: 21.693192971664143
coat
KL joint: 16.43131129532984
KL marginal: 29.42408069558971
sandal
KL joint: 18.11249506145907
KL marginal: 43.89401473305362
shirt
KL joint: 14.009425283623434
KL marginal: 11.533652712642787
sneaker
KL joint: 24.03513190388634
KL marginal: 58.191590372146806
bag
KL joint: 17.707023228512405
KL marginal: 34.54752332904317
boot
KL joint: 21.918086379161664
KL marginal: 50.38283941276504
Outer loop pullover
trouser
KL joint: 17.098063284723622
KL marginal: 41.86391770552747
pullover
KL joint: 0.08035840202867028
KL marginal: 26.21745876800793
dress
KL joint: 10.064926724454747
KL marginal: 21.693192971664143
coat
KL joint: 3.284572523378163
KL marginal: 29.42408069558971
sandal
KL joint: 12.093500256222736
KL marginal: 43.89401473305362
shirt
KL joint: 2.4495886

In [67]:
joblib.dump(div_joint, 'div_joint.pkl')
joblib.dump(div_margin, 'div_margin.pkl')

['div_margin.pkl']

In [94]:
for test_j in name_list[1:]:
    print('Outer loop:', test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['rec'][test_j]
    div_margin_train_test = div_margin_train[test_j]
    
    for i in name_list[1:]:
        print(i)
        abnormal_train_i = latent_dict_train['rec'][i]

        # Calculating for marginal divergence
        left = abnormal_train_i
        right = abnormal_test
        div_margin_train_test[i] = .5 * (skl_estimator(left, right) + 
                                         skl_estimator(right, left))
        print('KL marginal:', div_margin_train_test[i]) 

Outer loop: trouser
trouser
KL marginal: -0.036770070745401284
pullover
KL marginal: 45.43635870878137
dress
KL marginal: 26.700854283916755
coat
KL marginal: 40.08281641175711
sandal
KL marginal: 53.35113759423193
shirt
KL marginal: 38.74991326563158
sneaker
KL marginal: 67.38192686445177
bag
KL marginal: 45.48885681400773
boot
KL marginal: 61.329696317545555
Outer loop: pullover
trouser
KL marginal: 45.22831721190994
pullover
KL marginal: 0.1149328889066239
dress
KL marginal: 28.52068541661654
coat
KL marginal: 6.919252942576798
sandal
KL marginal: 35.7676349735828
shirt
KL marginal: 7.575748681272274
sneaker
KL marginal: 49.86269655853987
bag
KL marginal: 24.741496590252755
boot
KL marginal: 38.95588926063979
Outer loop: dress
trouser
KL marginal: 27.06221193824782
pullover
KL marginal: 27.253535212968274
dress
KL marginal: 0.11878140140208471
coat
KL marginal: 21.34300989025692
sandal
KL marginal: 39.13950828105992
shirt
KL marginal: 19.21375020550574
sneaker
KL marginal: 53.008932

In [95]:
joblib.dump(div_margin_train, 'div_margin_train.pkl')

['div_margin_train.pkl']

## Calculate for $KL(P(normal test) || P(abnormal test_{i}))$

In [84]:
margin_test_for_tshirt = []

In [85]:
normal_test = latent_dict_test['rec']['tshirt']

In [86]:
for test_j in name_list:
    print(test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['rec'][test_j]
    left = normal_test
    right = abnormal_test
    div = .5 * (skl_estimator(left, right) + 
                skl_estimator(right, left))
    margin_test_for_tshirt.append(div)
    print(div)

tshirt
-2.5883804237990122
trouser
38.455060035527545
pullover
24.675192972022522
dress
20.00445290988972
coat
26.38985571566586
sandal
40.845855219701484
shirt
10.370323401896218
sneaker
57.960567605358165
bag
33.69345477945552
boot
47.475840081542266


In [87]:
joblib.dump(margin_test_for_tshirt, 'margin_test_for_tshirt.pkl')

['margin_test_for_tshirt.pkl']

# Calculating Joint KL Divergence for One Class Model

In [74]:
div_joint_oc = {k: {} for k in name_list[1:]}
div_margin_oc = {k: {} for k in name_list[1:]}

# Fix normal train, normal test.
normal_train = latent_dict_train['oc']['tshirt']
normal_test = latent_dict_test['oc']['tshirt']

In [75]:
for test_j in name_list[1:]:
    print('Outer loop', test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['oc'][test_j]
    div_joint_test_oc = div_joint_oc[test_j]
    div_margin_test_oc = div_margin_oc[test_j]
    
    for i in name_list[1:]:
        print(i)
        abnormal_train_i = latent_dict_train['oc'][i]

        # Calculating for joint divergence
        left_joint = np.r_[normal_train, abnormal_train_i]
        right_joint = np.r_[normal_test, abnormal_test]
        div_joint_test_oc[i] = .5 * (skl_estimator(left_joint, right_joint) + 
                                     skl_estimator(right_joint, left_joint))
        print('KL joint:', div_joint_test_oc[i])

        # Calculating for marginal divergence
        left_margin = normal_train
        right_margin = abnormal_train_i
        div_margin_test_oc[i] = .5 * (skl_estimator(left_margin, right_margin) + 
                                      skl_estimator(right_margin, left_margin))
        print('KL marginal:', div_margin_test_oc[i]) 

Outer loop trouser
trouser
KL joint: 0.1571087161617828
KL marginal: 26.376290702408376
pullover
KL joint: 9.578044624604189
KL marginal: 12.567945190595136
dress
KL joint: 8.435332939404379
KL marginal: 11.093738491689127
coat
KL joint: 9.885738114828666
KL marginal: 13.631381574734245
sandal
KL joint: 11.138274194993901
KL marginal: 20.81608432288214
shirt
KL joint: 8.208346152621159
KL marginal: 5.8416629878152335
sneaker
KL joint: 16.782071631894926
KL marginal: 33.400634372643445
bag
KL joint: 13.058282679278186
KL marginal: 25.45356919663699
boot
KL joint: 20.68067429002034
KL marginal: 50.39134410773303
Outer loop pullover
trouser
KL joint: 10.937138640217977
KL marginal: 26.376290702408376
pullover
KL joint: 0.09322807240125774
KL marginal: 12.567945190595136
dress
KL joint: 5.203458860230839
KL marginal: 11.093738491689127
coat
KL joint: 2.1962844761785565
KL marginal: 13.631381574734245
sandal
KL joint: 6.517230272561263
KL marginal: 20.81608432288214
shirt
KL joint: 1.485781

In [76]:
joblib.dump(div_joint_oc, 'div_joint_oc.pkl')
joblib.dump(div_margin_oc, 'div_margin_oc.pkl')

['div_margin_oc.pkl']

In [88]:
margin_test_for_tshirt_oc = []
normal_test = latent_dict_test['oc']['tshirt']

for test_j in name_list:
    print(test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['oc'][test_j]
    left = normal_test
    right = abnormal_test
    div = .5 * (skl_estimator(left, right) + 
                skl_estimator(right, left))
    margin_test_for_tshirt_oc.append(div)
    print(div)

tshirt
-1.4190536483133809
trouser
24.094027436421015
pullover
10.81146036180348
dress
9.04938350952607
coat
11.998128268035167
sandal
17.660845482728046
shirt
4.0038609395016245
sneaker
31.433995234233155
bag
21.629337258628134
boot
55.1412669253435


In [89]:
joblib.dump(margin_test_for_tshirt_oc, 'margin_test_for_tshirt_oc.pkl')

['margin_test_for_tshirt_oc.pkl']

In [96]:
div_margin_train_oc = {k: {} for k in name_list[1:]}

In [97]:
for test_j in name_list[1:]:
    print('Outer loop:', test_j)
    # Fix abnormal test
    abnormal_test = latent_dict_test['oc'][test_j]
    div_margin_train_test_oc = div_margin_train_oc[test_j]
    
    for i in name_list[1:]:
        print(i)
        abnormal_train_i = latent_dict_train['oc'][i]

        # Calculating for marginal divergence
        left = abnormal_train_i
        right = abnormal_test
        div_margin_train_test_oc[i] = .5 * (skl_estimator(left, right) + 
                                            skl_estimator(right, left))
        print('KL marginal:', div_margin_train_test_oc[i]) 

Outer loop: trouser
trouser
KL marginal: 0.3418829994027215
pullover
KL marginal: 25.390113256006124
dress
KL marginal: 20.58516267392325
coat
KL marginal: 25.461273526462353
sandal
KL marginal: 31.919051914129927
shirt
KL marginal: 23.427027492852027
sneaker
KL marginal: 43.68733559323206
bag
KL marginal: 31.69053871488375
boot
KL marginal: 50.77386415715404
Outer loop: pullover
trouser
KL marginal: 27.402429809423914
pullover
KL marginal: 0.25660605251210544
dress
KL marginal: 14.22540254524386
coat
KL marginal: 5.167763583311194
sandal
KL marginal: 18.995829747596517
shirt
KL marginal: 4.645605052355478
sneaker
KL marginal: 30.751481574396916
bag
KL marginal: 21.838298646152694
boot
KL marginal: 47.187591009001565
Outer loop: dress
trouser
KL marginal: 21.556589480464666
pullover
KL marginal: 13.442165990367073
dress
KL marginal: 0.09925495209153912
coat
KL marginal: 12.766380432334238
sandal
KL marginal: 18.80190898863973
shirt
KL marginal: 9.920552384088825
sneaker
KL marginal: 30

In [98]:
joblib.dump(div_margin_train_oc, 'div_margin_train_oc.pkl')

['div_margin_train_oc.pkl']